# Find all must support elements in StructureDefinitions

- for hard coded path search all SD (in YAML) for must support = true elements
- list em in a md table format in std out
- update em with with teh USCDI extension and write back to YAML

In [1]:
from pathlib import Path
from json import load, dumps
from yaml import load as y_load, dump as y_dump, FullLoader
from fhir.resources import construct_fhir_element
from pprint import pprint
my_path = Path('/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml')
snapshot_path =  Path('/Users/ehaas/Documents/FHIR/US-Core/output')
my_path

PosixPath('/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml')

In [11]:
ms_id_map= {'Observation.component:Concentration.valueQuantity': 'Observation.component:Concentration.value[x]',
 'Observation.component:Concentration.valueQuantity.code': 'Observation.component:Concentration.value[x].code',
 'Observation.component:Concentration.valueQuantity.system': 'Observation.component:Concentration.value[x].system',
 'Observation.component:Concentration.valueQuantity.unit': 'Observation.component:Concentration.value[x].unit',
 'Observation.component:Concentration.valueQuantity.value': 'Observation.component:Concentration.value[x].value',
 'Observation.component:FlowRate.valueQuantity': 'Observation.component:FlowRate.value[x]',
 'Observation.component:FlowRate.valueQuantity.code': 'Observation.component:FlowRate.value[x].code',
 'Observation.component:FlowRate.valueQuantity.system': 'Observation.component:FlowRate.value[x].system',
 'Observation.component:FlowRate.valueQuantity.unit': 'Observation.component:FlowRate.value[x].unit',
 'Observation.component:FlowRate.valueQuantity.value': 'Observation.component:FlowRate.value[x].value',
 'Observation.component:diastolic.valueQuantity': 'Observation.component:diastolic.value[x]',
 'Observation.component:diastolic.valueQuantity.code': 'Observation.component:diastolic.value[x].code',
 'Observation.component:diastolic.valueQuantity.system': 'Observation.component:diastolic.value[x].system',
 'Observation.component:diastolic.valueQuantity.unit': 'Observation.component:diastolic.value[x].unit',
 'Observation.component:diastolic.valueQuantity.value': 'Observation.component:diastolic.value[x].value',
 'Observation.component:industry.valueCodeableConcept': 'Observation.component:industry.value[x]',
 'Observation.component:systolic.valueQuantity': 'Observation.component:systolic.value[x]',
 'Observation.component:systolic.valueQuantity.code': 'Observation.component:systolic.value[x].code',
 'Observation.component:systolic.valueQuantity.system': 'Observation.component:systolic.value[x].system',
 'Observation.component:systolic.valueQuantity.unit': 'Observation.component:systolic.value[x].unit',
 'Observation.component:systolic.valueQuantity.value': 'Observation.component:systolic.value[x].value',
 'Observation.effectiveDateTime': 'Observation.effective[x]:effectiveDateTime',
 'Observation.effectivePeriod': 'Observation.effective[x]:effectivePeriod',
 'Observation.valueCodeableConcept': 'Observation.value[x]:valueCodeableConcept',
 'Observation.valueQuantity': 'Observation.value[x]:valueQuantity',
 'Observation.valueQuantity.code': 'Observation.value[x]:valueQuantity.code',
 'Observation.valueQuantity.system': 'Observation.value[x]:valueQuantity.system',
 'Observation.valueQuantity.unit': 'Observation.value[x]:valueQuantity.unit',
 'Observation.valueQuantity.value': 'Observation.value[x]:valueQuantity.value'}

exclude_list = ['us-core-location','us-core-organization','us-core-practitioner','us-core-practitionerrole','us-core-direct','us-core-extension-questionnaire-uri','us-core-jurisdiction', 'uscdi-requirement']
include_list = ['us-core-location']  # for updating a single resource

uscdi_whitelist = [
'Patient.telecom',
'Patient.communication',
'Patient.extension:race',
'Patient.extension:ethnicity',
'Patient.extension:tribalAffiliation',
'Patient.extension:birthsex',
'Patient.extension:genderIdentity',
'Patient.deceased[x]',
'Patient.address.use',
'Patient.name.use',
'Patient.name.suffix',
'ServiceRequest.reasonCode',
'ServiceRequest.reasonReference',
'MedicationRequest.reasonCode',
'MedicationRequest.reasonReference',
'Procedure.basedOn',
'DocumentReference.category:uscore',
]

count = 0
cant_match_path = []
my_ext = {
            "url": "http://hl7.org/fhir/us/core/StructureDefinition/uscdi-requirement",
            "valueBoolean": True
        }

for enum,i in enumerate(my_path.glob('Struct*.yml')):

    obj = y_load(i.read_text(),Loader=FullLoader) #dict

    if include_list and obj['id'] not in include_list:  # for updating a single resource
        continue
    # elif obj['id'] in exclude_list:   # for updating all resources except those in the exclude list
    #     continue
    print(enum,i,obj['type'])
    print(obj['id'])
    snapshot_file = snapshot_path / f'{i.stem}.json'
    snapshot = load(snapshot_file.open())
    

    #find choice type elements in diff
    for e in obj['differential']['element']:
        try:
            if  e['id'] in uscdi_whitelist or e['mustSupport']:
                count += 1
                # print(f"e['id'] = {e['id']}")
                # add extension to the element
                # print(dumps(e, indent=2))
                try:
                    e['extension'].append(my_ext)
                except KeyError:
                    e['extension'] = [my_ext]
                # need to add to the short from the snapshot
                #get snapshot_short
                try:
                    snapshot_short = (s for s in snapshot['snapshot']['element'] if s['id'] == e['id']).__next__()['short']
                except StopIteration:
                    try:
                        snapshot_short = (s for s in snapshot['snapshot']['element'] if s['id'] == ms_id_map[e['id']]).__next__()['short']
                    except StopIteration:
                        cant_match_path.append((e['id'],e['path']))
                print(".", end='')
                # print(snapshot_short)
                try:
                    e['short'] = f"(USCDI Requirement) {snapshot_short}"
                except KeyError:
                    e['short'] = '(USCDI Requirement)'

                # print(yaml.dump(e, indent=2))
                # list the choices
                # for snapshot in obj['snapshot']['element']:
                #     if e['path'] == snapshot['path']:
                #     # print(snapshot['path'])
                #         types = [type['code'] for type in snapshot['type'] if type['code'] in 
                #                  ['code',"Coding","CodeableConcept"]]
                #         if types and snapshot['binding']['strength']=='extensible':
                #             print(f"{obj['title']}|{e['path']}|e min: {e['min']}|{types}|{snapshot['binding']['strength']}")
                #         break
                # print( f'writing file to {i}')
                # fhir_obj = construct_fhir_element(obj['resourceType'], obj) # sort elements and validate
                # # i.write_text(fhir_obj.yaml(indent=True))

                # print(fhir_obj.yaml(indent=True))
                

        except KeyError:
            pass
    print()
    print( f"writing {obj['id']} to {i}\n\n")
    fhir_obj = construct_fhir_element(obj['resourceType'], obj) # sort elements and validate
    # i.write_text(fhir_obj.yaml(indent=True))

    print(fhir_obj.yaml(indent=True))
    # if count > 25 : break
print(f'{enum} profiles with {count} mustSupport elements added to the differential')

print(f'cant_match_path = {set(cant_match_path)}')

19 /Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml/StructureDefinition-us-core-location.yml Location
us-core-location
.........
writing us-core-location to /Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml/StructureDefinition-us-core-location.yml


resourceType: StructureDefinition
id: us-core-location
url: http://hl7.org/fhir/us/core/StructureDefinition/us-core-location
name: USCoreLocation
title: US Core Location Profile
status: active
experimental: false
date: 2022-04-20
description: This profile sets minimum expectations for the Location resource for
  recording, searching for and fetching a Location associated with a patient, provider
  or organization to promote interoperability and adoption through common implementation.  It
  identifies which core elements, extensions, vocabularies, and value sets **SHALL**
  be present and constrains the way the elements are used when using the profile.  It
  provides the floor for standards development for specific use cases.
